**Music Recommendation System: Emotion Detection Model Training**

  - Emotion Detection: EfficientNetB0 CNN trained on FER-2013 dataset
  - User Activity Input: 5 activities (Studying, Working, Relaxing, Exercising, Commuting)
                   

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix
import time


2025-07-15 13:38:07.196682: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 13:38:07.198811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-15 13:38:07.241519: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-15 13:38:07.243302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-15 13:38:08.534239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Configuration

In [4]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: []


In [3]:
#Forces TensorFlow to use only CPU, preventing GPU memory issues
#Disables GPU visibility to ensure CPU-only training

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
tf.config.set_visible_devices([], 'GPU')    # Ensure no GPU devices are visible

# System information
print(f"TensorFlow version: {tf.__version__}")
print(f"Available devices: {tf.config.list_physical_devices()}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

TensorFlow version: 2.13.0
Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
GPU Available: False


In [5]:
# Set paths
BASE_DIR = Path("/app")
DATA_DIR = BASE_DIR / "data" / "processed" / "FC211002_Nethmi"   # path to processed dataset
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR = DATA_DIR / "test"

MODEL_DIR = BASE_DIR / "model" / "FC211002_Nethmi"   # Output model directory
MODEL_DIR.mkdir(parents=True, exist_ok=True)


In [6]:
# Define Constants
IMG_SIZE = (48, 48) #processed images are 48x48
BATCH_SIZE = 32  # Small batch size for CPU training efficiency
EPOCHS = 20

NUM_CLASSES = 5
CLASS_NAMES = ['angry', 'happy', 'sad', 'stressed', 'neutral']

LEARNING_RATE = 0.001  # learning rate for Adam optimizer

In [7]:
print(f"\nConfiguration:")
print(f"Data directory: {DATA_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Image size: {IMG_SIZE}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Batch size: {BATCH_SIZE}")



Configuration:
Data directory: /app/data/processed/FC211002_Nethmi
Model directory: /app/model/FC211002_Nethmi
Image size: (48, 48)
Number of classes: 5
Batch size: 32


## Data Loading

In [ ]:
def create_data_generators():
    
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,           # Normalize pixel values to [0,1]
        rotation_range=20,           # Random rotation up to 20 degrees
        width_shift_range=0.1,       # Random horizontal shift
        height_shift_range=0.1,      # Random vertical shift
        horizontal_flip=True,        # Random horizontal flip
        zoom_range=0.1,             # Random zoom in/out
        validation_split=0.2         # Reserve 20% for validation
    )
    
    # Validation data generator (no augmentation)
    val_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )
    
    # Load training data
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR / "train",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',    # One-hot encoded labels
        subset='training',           # Use 80% for training
        shuffle=True,               # Shuffle data for better training
        color_mode='rgb'            # RGB images (3 channels)
    )
    
    # Load validation data
    val_generator = val_datagen.flow_from_directory(
        DATA_DIR / "train",          # Use same directory but different subset
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',         # Use 20% for validation
        shuffle=False,              # Don't shuffle validation data
        color_mode='rgb'
    )
    
    return train_generator, val_generator

